# Pruning Example
This notebook demonstrates how to use the pruning methods from this tool to compress a model. 

The example uses the MNIST dataset and a simple CNN model. The model is trained and then pruned using the methods in this tool. The pruned model is then evaluated on the test set to see how well it performs.

In [1]:
#Define pytorch model
import os
import sys
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import torch.optim as optim
import transformers
import importlib
import inspect
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Add thesis package to path
sys.path.append("../")

import src.general as general

In [2]:
# Set device
no_cuda = False
use_cuda = not no_cuda and torch.cuda.is_available()
print(f"Using cuda: {use_cuda}")

device = torch.device("cuda" if use_cuda else "cpu")

Using cuda: False


### Load model

In [3]:
model_state = "../models/mnist.pt"
model_class = "models.mnist"

# Import the module classes
module = importlib.import_module(model_class)
classes = general.get_module_classes(module)
for cls in classes:
    globals()[cls.__name__] = cls

# Get device
device = general.get_device()

# Load the model
model = torch.load(model_state, map_location=torch.device(device))

Using cuda: False


Prepare data loaders for training and testing.

In [4]:
# Load MNIST dataset
batch_size = 64
test_batch_size = 64
use_cuda = False


kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
mnist_transform = transform=transforms.Compose([
                           transforms.ToTensor()
                       ])
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=mnist_transform,),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, download=True, transform=mnist_transform,),
    batch_size=test_batch_size, shuffle=True, **kwargs)

Define train and test methods.

In [5]:
# Hyperparameters
epochs = 3
lr = 0.01
momentum = 0.5
log_interval = 100

In [6]:
from models.mnist import MnistModel
from src.general import train, test

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
criterion = F.nll_loss

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, criterion, optimizer, epoch)
    test(model, device, test_loader, criterion, epoch)

Train Epoch: 1 [0/60000 (0%)]	 Distillation Loss: 0.0996
Train Epoch: 1 [6400/60000 (11%)]	 Distillation Loss: 0.0621
Train Epoch: 1 [12800/60000 (21%)]	 Distillation Loss: 0.0587
Train Epoch: 1 [19200/60000 (32%)]	 Distillation Loss: 0.0725
Train Epoch: 1 [25600/60000 (43%)]	 Distillation Loss: 0.0051
Train Epoch: 1 [32000/60000 (53%)]	 Distillation Loss: 0.0316
Train Epoch: 1 [38400/60000 (64%)]	 Distillation Loss: 0.1369
Train Epoch: 1 [44800/60000 (75%)]	 Distillation Loss: 0.0556
Train Epoch: 1 [51200/60000 (85%)]	 Distillation Loss: 0.0113
Train Epoch: 1 [57600/60000 (96%)]	 Distillation Loss: 0.1007
========================================= PERFORMANCE =============================================
Train Set: Average loss: 0.0288
Elapsed time = 20047.72 milliseconds (21.37 per batch)
Test Epoch: 1 [0/10000 (0%)]	Loss: 0.021109
Test Epoch: 1 [6400/10000 (64%)]	Loss: 0.157729
Test Epoch: 1 [2496/10000 (99%)]	Loss: 0.050407
========================================= PERFORMANCE =====

In [ ]:
save_model = True
save_model_path = "../models/mnist.pt"

if save_model:
    torch.save(model, save_model_path)

## Pruning

In [ ]:
# Test model performance before pruning
print("Before pruning")
test(model, device, test_loader)

# Define the parameters to prune
parameters_to_prune = (
    (model.conv1, 'weight'),
    (model.conv2, 'weight'),
    (model.fc1, 'weight'),
    (model.fc2, 'weight'),
)

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.95,
)

# Test model performance after pruning
print("After pruning")
test(model, device, test_loader)

# Print number of parameters in model
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')


# Print number of parameters pruned
total_pruned_params = sum(p.numel() for p in model.parameters() if hasattr(p, 'mask'))
print(f'{total_pruned_params:,} parameters pruned ({100 * total_pruned_params / total_params:.2f}% pruned)')

# Print number of parameters remaining
total_unpruned_params = sum(p.numel() for p in model.parameters() if not hasattr(p, 'mask'))
print(f'{total_unpruned_params:,} parameters unpruned ({100 * total_unpruned_params / total_params:.2f}% unpruned)')


Before pruning


TypeError: test() missing 2 required positional arguments: 'criterion' and 'epoch'